In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm

In [2]:
N_arms = 8
N_resources = 4

base = ['F']*N_resources + ['0']*(N_arms-N_resources)
# find all permutations of variants
all_perms = list(itertools.permutations(base))
# remove effective duplicates
variants = []
for perm in tqdm(all_perms):
    # all rotations of a variant are equivalent and so with reflections
    rotations = [perm]
    for i in range(1, N_arms):
        rotations.append(perm[i:] + perm[:i])
    reflections = []
    for rotation in rotations:
        reflections.append(rotation[::-1])
    all_versions = rotations + reflections
    # check if variant is already in list
    all_version_in_list = [x in variants for x in all_versions]
    if not any(all_version_in_list):
        variants.append(perm)

100%|██████████| 40320/40320 [00:00<00:00, 90864.24it/s]


In [3]:
variants = [''.join(variant) for variant in variants]
print('Found {} variants'.format(len(variants)))
print('Variants:')
for variant in variants:
    print('\'{}\''.format(variant))

Found 8 variants
Variants:
'FFFF0000'
'FFF0F000'
'FFF00F00'
'FF0FF000'
'FF0F0F00'
'FF0F00F0'
'FF00FF00'
'F0F0F0F0'


In [4]:
def variant_to_points(variant,arm_length=1):
    points = []
    for i in range(len(variant)):
        if variant[i] == 'F':
            x = np.cos(i * np.pi / 4) * arm_length
            y = np.sin(i * np.pi / 4) * arm_length
            points.append((x, y)) 
    return points

def get_euclidean_matrix(points):
    matrix = np.zeros((len(points),len(points)))
    for i in range(len(points)):
        for j in range(len(points)):
            matrix[i,j] = np.sqrt((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2)
    return matrix

def variant_to_spatial_variance(variant):
    points = variant_to_points(variant)
    matrix = get_euclidean_matrix(points)
    return np.mean(matrix[np.triu_indices(len(points),1)])


def variant_to_left_turns(variant):
    # find the indices of the F's
    indices = []
    for i in range(len(variant)):
        if variant[i] == 'F':
            indices.append(i)
    # for each index, find number of left shifts to the other indices assuming circular
    left_turns = []
    for i in range(len(indices)):
        for_i_th = []
        for j in range(len(indices)):
            for_i_th.append((indices[j] - indices[i]) % len(variant))
        left_turns.append(for_i_th)
    return left_turns

def variant_to_right_turns(variant):
    # find the indices of the F's
    indices = []
    for i in range(len(variant)):
        if variant[i] == 'F':
            indices.append(i)
    # for each index, find number of left shifts to the other indices assuming circular
    right_turns = []
    for i in range(len(indices)):
        for_i_th = []
        for j in range(len(indices)):
            for_i_th.append((indices[i] - indices[j]) % len(variant))
        right_turns.append(for_i_th)
    return right_turns

def variant_to_closest_paths(variant):
    left_turns = np.array(variant_to_left_turns(variant))
    right_turns = np.array(variant_to_right_turns(variant))
    # find which turns are the smallest
    left_smaller = left_turns < right_turns
    closer_turns = np.zeros(left_turns.shape)
    closer_turns[left_smaller] = left_turns[left_smaller]
    closer_turns[~left_smaller] = -right_turns[~left_smaller]
    return closer_turns

def variant_to_mean_distance(variant):
    paths = np.abs(variant_to_closest_paths(variant))
    # remove the 0's
    paths = paths[paths > 0]
    # we want to maximize the lower bound of the distance and maximize the upper bound of the distance
    # count how many are more than the mean
    return np.sum(paths > np.mean(paths))/np.sum(paths < np.mean(paths))

def path_to_dice_matrix(path):
    matrix = np.zeros((len(path),len(path)))
    for i in range(len(path)):
        for j in range(len(path)):
            pi = path[i][path[i] != 0]
            pj = path[j][path[j] != 0]
            # count how many elements are in the intersection and calculate the dice coefficient
            matrix[i,j] = len(set(pi).intersection(pj)) / len(set(pi).union(pj))
    return matrix

def variant_to_mean_dice_closest(variant):
    paths = variant_to_closest_paths(variant)
    matrix = path_to_dice_matrix(paths)
    return np.mean(matrix[np.triu_indices(len(paths),1)])

In [6]:
print('Variant','\tSpatial Variance','\tPostural heuristic similarity')
for variant in variants:
    print(variant, "\t{:.5f}".format(variant_to_spatial_variance(variant)), "\t\t{:.5f}".format(variant_to_mean_dice_closest(variant)))

Variant 	Spatial Variance 	Postural heuristic similarity
FFFF0000 	1.16205 		0.31667
FFF0F000 	1.36782 		0.16667
FFF00F00 	1.44008 		0.16667
FF0FF000 	1.44008 		0.16667
FF0F0F00 	1.54822 		0.16667
FF0F00F0 	1.52285 		0.31667
FF00FF00 	1.53771 		0.46667
F0F0F0F0 	1.60948 		1.00000


In [11]:
for variant in variants:
    print(variant)

FFFF0000
FFF0F000
FFF00F00
FF0FF000
FF0F0F00
FF0F00F0
FF00FF00
F0F0F0F0
